In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports 

In [ ]:
import warnings
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn import tree
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

warnings.filterwarnings('ignore')

In [ ]:
data = '/kaggle/input/car-evaluation-data-set/car_evaluation.csv'

df = pd.read_csv(data, header=None)

# Renaming cols

In [ ]:
col_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

df.columns = col_names

col_names

# Removing unnecessary cols

In [ ]:
X = df.drop(['class'], axis=1)

Y = df['class']

In [ ]:
X.head()

In [ ]:
Y.head()

# Setting up Kfold and encoding categorical variables

In [ ]:
kfold = KFold(3, True, 1)

In [ ]:
encoder = ce.OrdinalEncoder(cols=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety'])
X = encoder.fit_transform(X)
#X_test = encoder.transform(X_test)

In [ ]:
X = X.to_numpy()
Y = Y.to_numpy()

# Training and testing over the multiple splits created using Kfold

In [ ]:
i=1
for train, test in kfold.split(X):
    
    X_train = X[train]
    X_test = X[test]
    Y_train = Y[train]
    Y_test = Y[test]
    
    clf_gini = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=0)

    clf_gini.fit(X_train, Y_train)
    y_pred_gini = clf_gini.predict(X_test)
    
    print('Split number: '+str(i))
    i=i+1
    print('Model accuracy score with criterion gini index: {0:0.4f}'. format(accuracy_score(Y_test, y_pred_gini)))
    
    y_pred_train_gini = clf_gini.predict(X_train)
    y_pred_train_gini
    
    print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(Y_train, y_pred_train_gini)))
    
    print('Training set score: {:.4f}'.format(clf_gini.score(X_train, Y_train)))
    print('Test set score: {:.4f}'.format(clf_gini.score(X_test, Y_test)))
    
    plt.figure(figsize=(12,12))
    #col_names = ['unacc', 'acc']
    tree.plot_tree(clf_gini.fit(X_train, Y_train), class_names = list(df['class'].unique()))
    
    print(classification_report(Y_train, y_pred_train_gini, target_names = list(df['class'].unique())))